# Analyse de la qualité des données du fichier Ventes_Q1_2025.csv


## Résumé des anomalies observées

## Champs ville
- 17% de valeurs manquantes (83% de complétude)
- 25% de validité (75% de villes erronnées, similaires à clients)
- 8% de cohérence avec la table Client (les noms de ville des clients ne sont pas reportés dans la table vente)

## Champ date
- 25% des lignes correspondent à la période Q1 (1er trimestre)

## Champ prix
- 9% de prix négatifs (91% de prix valides)

## Champ quantité
- 2,2% de quantités négatives (98% de quantités valides)

## Analyse croisée avec les fichiers Produit et MarketPlace
- les prix négatifs ne sont pas dû à un transfert d'anomalie depuis ce fichier
- les chiffres d'affaires négatifs ne correspondent pas à des produits retournés sur le canal Web (0,7% de corrélation)

## Analyse de la cohérence des client_id entre Ventes et Clients_Master
- 100% des client_id provenant de la table ventes sont bien présents dans la table client
- 0,6 % de cohérence de la date de vente avec la date de dernière activité (Client_Master)

## Analyse de la cohérence du chiffre d'affaire
- CA Total 2025 : 443,777.55 € alors que l'énoncé présente 45 millions par an.
Le fichier devrait donc contenir environ 10 fois plus de lignes


### Points positifs
- ✅ **Aucun doublon** détecté dans le dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ventes = pd.read_csv('../data/raw/Ventes_Q1_2025.csv')

In [3]:
print("AUDIT DATAFRAME VENTES 1er TRIMESTRE 2025")

display(df_ventes.head())
display(df_ventes.info())

print("\n---Taux de valeurs manquantes (%)---")
missing_percentage = df_ventes.isnull().sum() / len(df_ventes) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

print("\n--- Vérification des doublons ---")
clients_doublons = df_ventes.duplicated().sum()
print(f"Nombre total de doublons: {clients_doublons} / soit {clients_doublons/len(df_ventes) * 100:.2f}%")


print("\n--- Vérification des doublons sur le order_id ---")
nb_doublons_order_id = df_ventes['order_id'].duplicated().sum()
print(f"Nombre de order_id en doublon : {nb_doublons_order_id}")

AUDIT DATAFRAME VENTES 1er TRIMESTRE 2025


,order_id,date,client_id,produit,categorie,prix_unitaire,quantite,ville,canal,chiffre_affaires
0,1,2025-03-05,391,Batterie externe,Informatique,56.80,2,Saint ConstanceVille,Web,113.60
1,2,2025-01-26,261,Webcam HD,Informatique,146.42,-1,NaN,Web,-146.42
2,3,2025-08-03,461,Disque SSD,Informatique,11.44,3,Leleu-sur-Lopez,Boutique,34.32
3,4,2025-07-12,152,Batterie externe,Audio,138.78,2,Saint Lucie,Boutique,277.56
4,5,2025-04-18,107,Disque SSD,Informatqiue,9.88,5,Toussaint,Web,49.40


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          2000 non-null   int64  
 1   date              2000 non-null   object 
 2   client_id         2000 non-null   int64  
 3   produit           2000 non-null   object 
 4   categorie         2000 non-null   object 
 5   prix_unitaire     2000 non-null   float64
 6   quantite          2000 non-null   int64  
 7   ville             1651 non-null   object 
 8   canal             2000 non-null   object 
 9   chiffre_affaires  2000 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 156.4+ KB


None


---Taux de valeurs manquantes (%)---
ville    17.45
dtype: float64

--- Vérification des doublons ---
Nombre total de doublons: 0 / soit 0.00%

--- Vérification des doublons sur le order_id ---
Nombre de order_id en doublon : 0


In [4]:
print("\n--- Vérification de la cohérence temporelle de l'export ---")

# Convertir les dates si ce n'est pas déjà fait
df_ventes['date'] = pd.to_datetime(df_ventes['date'], errors='coerce')

# Identifier la période attendue d'après le nom du fichier
periode_attendue_debut = pd.Timestamp('2025-01-01')
periode_attendue_fin = pd.Timestamp('2025-03-31')
print(f"Période attendue d'après le fichier : Mars 2025")
print(f"  Du {periode_attendue_debut.strftime('%d/%m/%Y')} au {periode_attendue_fin.strftime('%d/%m/%Y')}")

# Analyser les dates réelles présentes dans le fichier
date_min = df_ventes['date'].min()
date_max = df_ventes['date'].max()

print(f"\nPériode réelle dans les données :")
print(f"  Date la plus ancienne : {date_min.strftime('%d/%m/%Y') if pd.notna(date_min) else 'NaT'}")
print(f"  Date la plus récente : {date_max.strftime('%d/%m/%Y') if pd.notna(date_max) else 'NaT'}")

# Identifier les lignes hors période
mask_avant = df_ventes['date'] < periode_attendue_debut
mask_apres = df_ventes['date'] > periode_attendue_fin
mask_hors_periode = mask_avant | mask_apres

nb_hors_periode = mask_hors_periode.sum()
nb_total = len(df_ventes)
pourcentage_hors_periode = (nb_hors_periode / nb_total) * 100

if nb_hors_periode > 0:
    print(f"\n⚠️ ANOMALIE DÉTECTÉE : {nb_hors_periode} lignes ({pourcentage_hors_periode:.2f}%) ont des dates hors période !")

    # Détailler les anomalies
    nb_avant = mask_avant.sum()
    nb_apres = mask_apres.sum()

    print(f"  - Dates avant janvier 2025 : {nb_avant}")
    print(f"  - Dates après mars 2025 : {nb_apres}")

    # Créer un DataFrame des anomalies
    df_hors_periode = df_ventes[mask_hors_periode].copy()
    df_hors_periode = df_hors_periode.sort_values('date')

    print("\n--- Exemples de lignes avec dates hors période ---")
    display(df_hors_periode[['order_id','date', 'produit', 'quantite', 'prix_unitaire','canal']].head(10))

else:
    print(f"\n✓ Toutes les dates correspondent à la période attendue (Mars 2025)")

    # Vérifier les dates NaT (non converties)
nb_nat = df_ventes['date'].isna().sum()
if nb_nat > 0:
    print(f"\n⚠️ {nb_nat} dates invalides (NaT) détectées")



--- Vérification de la cohérence temporelle de l'export ---
Période attendue d'après le fichier : Mars 2025
  Du 01/01/2025 au 31/03/2025

Période réelle dans les données :
  Date la plus ancienne : 29/10/2024
  Date la plus récente : 29/10/2025

⚠️ ANOMALIE DÉTECTÉE : 1494 lignes (74.70%) ont des dates hors période !
  - Dates avant janvier 2025 : 346
  - Dates après mars 2025 : 1148

--- Exemples de lignes avec dates hors période ---


,order_id,date,produit,quantite,prix_unitaire,canal
723,724,2024-10-29,Webcam HD,2,59.91,Boutique
354,355,2024-10-29,Chargeur rapide,2,120.56,Web
899,900,2024-10-29,Webcam HD,1,6.67,Boutique
1167,1168,2024-10-29,Enceinte Bluetooth,1,-103.78,Web
1788,1789,2024-10-29,Chargeur rapide,2,109.85,Web
764,765,2024-10-30,Clavier,0,136.57,Web
263,264,2024-10-30,Câble USB-C,1,61.01,Marketplace
1550,1551,2024-10-30,Disque SSD,2,-180.48,Web
520,521,2024-10-30,Clavier,1,88.22,Web
562,563,2024-10-30,Casque Bluetooth,1,153.48,Web


In [5]:
print("\n---Vérification des prix négatifs---")
nb_prix_negatifs = (df_ventes['prix_unitaire']<0).sum()
print("Nombre total de prix négatifs :",nb_prix_negatifs)
print(f"Pourcentage de prix négatifs : {nb_prix_negatifs/len(df_ventes) * 100:.2f}%")

lignes_prix_negatifs = df_ventes[df_ventes['prix_unitaire'] < 0]
print("\nExemples de lignes avec des prix négatifs :")
display(lignes_prix_negatifs.head(8))

# Identifier les lignes avec prix négatifs
mask_prix_negatifs = df_ventes['prix_unitaire'] < 0

# Compter les prix négatifs par canal
prix_negatifs_par_canal = df_ventes[mask_prix_negatifs]['canal'].value_counts()
print("\n--- Détail avec pourcentage ---")
for canal, count in prix_negatifs_par_canal.items():
    pourcentage = (count / mask_prix_negatifs.sum() * 100)
    print(f"{canal}: {count} prix négatifs ({pourcentage:.1f}%)")


---Vérification des prix négatifs---
Nombre total de prix négatifs : 183
Pourcentage de prix négatifs : 9.15%

Exemples de lignes avec des prix négatifs :


,order_id,date,client_id,produit,categorie,prix_unitaire,quantite,ville,canal,chiffre_affaires
7,8,2025-03-04,461,Webcam HD,Informatique,-15.16,1,Blin-la-Forêt,Web,-15.16
18,19,2024-12-27,496,Webcam HD,Informatique,-16.44,3,Dupuy-la-Forêt,Web,-49.32
44,45,2025-09-13,182,Écran 24 pouces,Informatique,-16.71,1,Clémentnec,Web,-16.71
48,49,2024-12-31,297,Enceinte Bluetooth,Informatique,-93.23,3,Lamy,Web,-279.69
53,54,2025-01-29,31,Enceinte Bluetooth,Informatqiue,-24.80,2,Salmon,Boutique,-49.60
59,60,2025-06-19,357,Clavier,Informatique,-67.01,1,Valette,Web,-67.01
75,76,2024-12-22,334,Disque SSD,Informatique,-161.49,1,Millet,Web,-161.49
94,95,2025-10-07,354,Webcam HD,Audio,-17.12,1,Regnier-sur-Nicolas,Web,-17.12



--- Détail avec pourcentage ---
Web: 113 prix négatifs (61.7%)
Boutique: 51 prix négatifs (27.9%)
Marketplace: 19 prix négatifs (10.4%)


In [6]:
print("\n---Vérification des quantités négatives---")
nb_prix_negatifs = (df_ventes['quantite']<0).sum()
print("Nombre total de quantités négatives :",nb_prix_negatifs)
print(f"Pourcentage de quantités négatives : {nb_prix_negatifs/len(df_ventes) * 100:.2f}%")

lignes_quantites_negatives = df_ventes[df_ventes['quantite'] < 0]
print("\nExemples de lignes avec des quantités négatives :")
display(lignes_quantites_negatives.head(8))

# Identifier les lignes avec prix négatifs
mask_quantite_negatives = df_ventes['quantite'] < 0

# Compter les prix négatifs par canal
prix_negatifs_par_canal = df_ventes[mask_quantite_negatives]['canal'].value_counts()
print("\n--- Détail avec pourcentage ---")
for canal, count in prix_negatifs_par_canal.items():
    pourcentage = (count / mask_quantite_negatives.sum() * 100)
    print(f"{canal}: {count} quantite negatives ({pourcentage:.1f}%)")


---Vérification des quantités négatives---
Nombre total de quantités négatives : 44
Pourcentage de quantités négatives : 2.20%

Exemples de lignes avec des quantités négatives :


,order_id,date,client_id,produit,categorie,prix_unitaire,quantite,ville,canal,chiffre_affaires
1,2,2025-01-26,261,Webcam HD,Informatique,146.42,-1,NaN,Web,-146.42
24,25,2025-09-08,213,Écran 24 pouces,Informatique,99.16,-1,NaN,Boutique,-99.16
104,105,2024-12-08,26,Câble USB-C,Audio,102.70,-1,Aubry-sur-Rivière,Web,-102.70
209,210,2025-09-22,347,Casque Bluetooth,Informatique,163.15,-1,Regnier-sur-Nicolas,Web,-163.15
266,267,2024-11-06,455,Souris,Informatique,90.89,-1,Colinnec,Web,-90.89
357,358,2025-04-11,449,Souris,Accessoire,90.29,-1,Sainte Isabelle,Boutique,-90.29
376,377,2025-07-14,200,Webcam HD,Audio,28.29,-1,Diallodan,Web,-28.29
404,405,2025-06-04,58,Batterie externe,Informatqiue,123.38,-1,Saint HenrietteVille,Web,-123.38



--- Détail avec pourcentage ---
Web: 29 quantite negatives (65.9%)
Boutique: 12 quantite negatives (27.3%)
Marketplace: 3 quantite negatives (6.8%)


In [7]:
print("\n--- Analyse des catégories de produits ---")

# Nombre de catégories uniques
nb_categories = df_ventes['categorie'].nunique()
print(f"Nombre de catégories uniques : {nb_categories}")
print("\n--- Répartition des catégories ---")
categories_count = df_ventes['categorie'].value_counts(dropna=False)

# Pourcentage par catégorie
categories_pct = df_ventes['categorie'].value_counts(dropna=False, normalize=True) * 100
for cat, pct in categories_pct.items():
    count = categories_count[cat]
    print(f"  {cat} : {count} ventes ({pct:.2f}%)")


--- Analyse des catégories de produits ---
Nombre de catégories uniques : 4

--- Répartition des catégories ---
  Informatique : 1159 ventes (57.95%)
  Audio : 410 ventes (20.50%)
  Informatqiue : 216 ventes (10.80%)
  Accessoire : 215 ventes (10.75%)


In [8]:
# Export des données anormales : prix et quantité négatifs pour analyse croisée avec l'export Marketplace
print("\n--- SAUVEGARDE DU FICHIER AVEC LES PRIX ET QUANTITES NEGATIFS ---")
mask_CA_negatif = df_ventes['chiffre_affaires'] < 0
df_ventes_ca_negatifs = df_ventes[mask_CA_negatif].copy()
df_ventes_ca_negatifs.to_csv('../data/processed/Ventes_Q1_2025_CA_negatifs.csv', index=False)


--- SAUVEGARDE DU FICHIER AVEC LES PRIX ET QUANTITES NEGATIFS ---


In [9]:
# import du df marketplace pour analyse croisée
df_marketplace_corrected = pd.read_csv('../data/processed/Marketplace_Export_Q1_2025.csv')

In [10]:
print("\n--- FILTRAGE CANAL WEB ---")

# Filtrer uniquement le canal "Web"
df_ventes_ca_negatifs_web = df_ventes_ca_negatifs[df_ventes_ca_negatifs['canal'] == 'Web'].copy()
print(f"Nombre de lignes avec canal 'Web' : {len(df_ventes_ca_negatifs_web)}")

print("\n--- ANALYSE DES CORRESPONDANCES AVEC MARKETPLACE ---")

# Fusionner avec correspondance order_id = order_id_market
df_fusion = df_ventes_ca_negatifs_web.merge(
    df_marketplace_corrected[['order_id_market', 'statut_retour']],
    left_on='order_id',
    right_on='order_id_market',
    how='left'
)

total = len(df_ventes_ca_negatifs_web)

# Créer le tableau des statistiques
stats = df_fusion['statut_retour'].value_counts(dropna=False)
pourcentages = (stats / total * 100).round(1)

df_resultats = pd.DataFrame({
    'Statut': stats.index.fillna('Aucune correspondance'),
    'Nombre': stats.values,
    'Pourcentage': pourcentages.values
})

print(f"Total analysé : {total} lignes\n")
display(df_resultats)


--- FILTRAGE CANAL WEB ---
Nombre de lignes avec canal 'Web' : 139

--- ANALYSE DES CORRESPONDANCES AVEC MARKETPLACE ---
Total analysé : 139 lignes



,Statut,Nombre,Pourcentage
0,Aucune correspondance,133,95.7
1,Livré,3,2.2
2,En attente,2,1.4
3,Retourné,1,0.7


In [11]:
# import du df Client_Master pour analyse croisée
df_clients_corrected = pd.read_csv('../data/processed/Clients_Master_corrected.csv')

In [12]:
print("\n--- CORRESPONDANCES CLIENT_ID ---")

# Clients uniques
clients_ventes = df_ventes['client_id'].nunique()
clients_trouves = df_ventes[df_ventes['client_id'].isin(df_clients_corrected['client_id'])]['client_id'].nunique()

pourcentage = (clients_trouves / clients_ventes * 100)

print(f"Client_id uniques dans ventes : {clients_ventes}")
print(f"Trouvés dans clients_corrected : {clients_trouves} ({pourcentage:.1f}%)")
print(f"Non trouvés : {clients_ventes - clients_trouves} ({100-pourcentage:.1f}%)")


--- CORRESPONDANCES CLIENT_ID ---
Client_id uniques dans ventes : 489
Trouvés dans clients_corrected : 489 (100.0%)
Non trouvés : 0 (0.0%)


In [13]:
print("\n--- CRÉATION DF_LAST_VENTES_CLIENT ---")
# Convertir le champ date en datetime
df_ventes['date'] = pd.to_datetime(df_ventes['date'])

# Trier par date décroissante puis garder la première occurrence de chaque client_id
df_last_ventes_client = df_ventes.sort_values('date', ascending=False).drop_duplicates(subset='client_id', keep='first').copy()

print(f"Nombre de client_id uniques : {len(df_last_ventes_client)}")


--- CRÉATION DF_LAST_VENTES_CLIENT ---
Nombre de client_id uniques : 489


In [14]:
print("\n--- ANALYSE CORRESPONDANCE DATE vs DERNIERE_ACTIVITE ---")

# Fusionner et préparer les données
df_comparaison = df_last_ventes_client.merge(
    df_clients_corrected[['client_id', 'derniere_activite']],
    on='client_id',
    how='inner'
)

df_comparaison['derniere_activite'] = pd.to_datetime(df_comparaison['derniere_activite'])
df_comparaison['date'] = pd.to_datetime(df_comparaison['date'])

# Calculer l'écart en jours
df_comparaison['ecart_jours'] = (df_comparaison['date'] - df_comparaison['derniere_activite']).dt.days

# Statistiques
dates_identiques = (df_comparaison['ecart_jours'] == 0).sum()
total = len(df_comparaison)
pourcentage = (dates_identiques / total * 100) if total > 0 else 0

print(f"Total de clients comparés : {total}")
print(f"Dates identiques : {dates_identiques} ({pourcentage:.1f}%)")
print(f"Dates différentes : {total - dates_identiques} ({100-pourcentage:.1f}%)")

print(f"\n--- Analyse des écarts ---")
print(f"Écart moyen : {df_comparaison['ecart_jours'].mean():.1f} jours")
print(f"Écart médian : {df_comparaison['ecart_jours'].median():.1f} jours")
print(f"Écart min : {df_comparaison['ecart_jours'].min()} jours")
print(f"Écart max : {df_comparaison['ecart_jours'].max()} jours")

# Afficher quelques exemples de différences
print("\n--- Exemples de différences ---")
df_differences = df_comparaison[df_comparaison['ecart_jours'] != 0][['client_id', 'date', 'derniere_activite', 'ecart_jours']].head(10)
display(df_differences)


--- ANALYSE CORRESPONDANCE DATE vs DERNIERE_ACTIVITE ---
Total de clients comparés : 489
Dates identiques : 3 (0.6%)
Dates différentes : 486 (99.4%)

--- Analyse des écarts ---
Écart moyen : 90.7 jours
Écart médian : 83.0 jours
Écart min : -314 jours
Écart max : 352 jours

--- Exemples de différences ---


,client_id,date,derniere_activite,ecart_jours
0,231,2025-10-29,2025-02-17,254
1,312,2025-10-29,2025-06-08,143
2,252,2025-10-29,2025-05-16,166
3,397,2025-10-29,2025-02-27,244
4,67,2025-10-28,2025-07-11,109
5,36,2025-10-28,2025-08-21,68
6,441,2025-10-28,2024-11-18,344
7,477,2025-10-28,2025-03-24,218
8,144,2025-10-28,2025-07-17,103
9,61,2025-10-28,2024-12-02,330


In [15]:
# copie du dataframe original
df_ventes_clean = df_ventes.copy()

In [16]:
print("\n--- VALIDATION DES NOMS DE VILLES ---")

import requests

def valider_villes_dataframe(df, colonne_ville='ville'):
    # Statistiques sur les valeurs vides
    nb_vides = df[colonne_ville].isna().sum()
    pourcentage_vides = (nb_vides / len(df) * 100)

    print(f"Valeurs vides : {nb_vides} soit {pourcentage_vides:.1f}%")

    # Villes uniques (hors valeurs vides)
    villes_uniques = df[colonne_ville].dropna().unique()

    villes_invalides = []
    villes_valides = []

    print(f"Validation de {len(villes_uniques)} villes uniques...")

    for i, ville in enumerate(villes_uniques, 1):

        url = f"https://geo.api.gouv.fr/communes?nom={ville}&limit=1"
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200 and len(response.json()) > 0:
                villes_valides.append(ville)
            else:
                villes_invalides.append(ville)
        except:
            villes_invalides.append(ville)

    print(f"\n✅ Villes valides : {len(villes_valides)} ({len(villes_valides)/len(villes_uniques)*100:.0f}%)")
    print(f"❌ Villes invalides : {len(villes_invalides)} ({len(villes_invalides)/len(villes_uniques)*100:.0f}%)")

    if villes_invalides:
        print(f"\nVilles invalides : {villes_invalides}")

        mask_invalides = df[colonne_ville].isin(villes_invalides)
        nb_lignes = mask_invalides.sum()

        print(f"Lignes affectées : {nb_lignes} ({nb_lignes/len(df)*100:.1f}%)")

        df_invalides = df[mask_invalides].copy()
        display(df_invalides[['client_id', colonne_ville]].head(10))

        return villes_valides, villes_invalides, df_invalides

    return villes_valides, villes_invalides, None

villes_valides, villes_invalides, df_villes_invalides = valider_villes_dataframe(df_ventes)



--- VALIDATION DES NOMS DE VILLES ---
Valeurs vides : 349 soit 17.4%
Validation de 351 villes uniques...

✅ Villes valides : 87 (25%)
❌ Villes invalides : 264 (75%)

Villes invalides : ['Saint ConstanceVille', 'Leleu-sur-Lopez', 'Pariss', 'Blin-la-Forêt', 'Sainte Margot-la-Forêt', 'Blanchet', 'Sainte Charlotte', 'Dumas', 'Dijoux-la-Forêt', 'Ruiz', 'Colinnec', 'Dupuy-la-Forêt', 'Aubry-sur-Rivière', 'Guillet', 'Le Gall-sur-Lemaire', 'Marchal-sur-Duhamel', 'Roger-sur-Mer', 'Bonnin-sur-Didier', 'Sainte Isabelle', 'Guillou', 'Millet', 'Deschampsboeuf', 'MoulinVille', 'Rolland-sur-Mer', 'Tanguy', 'Saint HenrietteVille', 'Sainte Josettedan', 'Clémentnec', 'Duhamel', 'Lamy', 'Sainte Laurencedan', 'Pereira', 'Saint Georges-sur-Mer', 'Grondin-la-Forêt', 'GomezBourg', 'Diaz', 'Lemoine', 'Sainte Aurélienec', 'Saint Richardnec', 'Saint Alain', 'RenaultBourg', 'Sanchez', 'Rémy-la-Forêt', 'Chauvetdan', 'Grégoireboeuf', 'GrégoireBourg', 'Bouchet-la-Forêt', 'Sanchezboeuf', 'LegrosBourg', 'Turpin', 'Lo

,client_id,ville
0,391,Saint ConstanceVille
2,461,Leleu-sur-Lopez
6,429,Pariss
7,461,Blin-la-Forêt
8,306,Sainte Margot-la-Forêt
9,239,Blanchet
10,295,Sainte Charlotte
11,21,Dumas
12,111,Dijoux-la-Forêt
13,311,Leleu-sur-Lopez


In [17]:
# import du dataframe Client original pour comparer la cohérence des erreurs de ville
df_clients_original = pd.read_csv('../data/raw/Clients_Master.csv')

In [18]:
print("\n--- CORRESPONDANCE GLOBALE VILLES CLIENT vs VENTES ---")

# Grouper les villes par client dans df_ventes
villes_par_client = df_ventes.groupby('client_id')['ville'].apply(set).to_dict()

# Vérifier la correspondance pour chaque client
correspondances = []
for _, row in df_clients_corrected.iterrows():
    client_id = row['client_id']
    ville_client = row['ville']

    if client_id in villes_par_client:
        ville_trouvee = ville_client in villes_par_client[client_id]
        correspondances.append(ville_trouvee)

nb_correct = sum(correspondances)
total = len(correspondances)
pourcentage_correct = (nb_correct / total * 100) if total > 0 else 0

print(f"Client_ID avec sa ville correspondante dans la table Vente : {nb_correct}/{total} ({pourcentage_correct:.1f}%)")
print(f"Clients_ID avec aucune ville correspondante dans la table Vente: {total - nb_correct}/{total} ({100 - pourcentage_correct:.1f}%)")


--- CORRESPONDANCE GLOBALE VILLES CLIENT vs VENTES ---
Client_ID avec sa ville correspondante dans la table Vente : 38/489 (7.8%)
Clients_ID avec aucune ville correspondante dans la table Vente: 451/489 (92.2%)


In [19]:
print("\n--- ANALYSE CORRESPONDANCE VILLES CLIENT vs VENTES (CLIENTS ACTIFS) ---")

# Convertir les dates en datetime
df_ventes['date'] = pd.to_datetime(df_ventes['date'])
df_clients_corrected['derniere_activite'] = pd.to_datetime(df_clients_corrected['derniere_activite'])

# Récupérer la période des ventes
date_min_ventes = df_ventes['date'].min()
date_max_ventes = df_ventes['date'].max()

print(f"Période des ventes : du {date_min_ventes.date()} au {date_max_ventes.date()}")

# Filtrer les clients dont derniere_activite est dans la période des ventes
df_clients_actifs = df_clients_corrected[
    (df_clients_corrected['derniere_activite'] >= date_min_ventes) &
    (df_clients_corrected['derniere_activite'] <= date_max_ventes)
].copy()

print(f"\nClients total dans df_clients_corrected : {len(df_clients_corrected)}")
print(f"Clients actifs dans la période : {len(df_clients_actifs)} ({len(df_clients_actifs)/len(df_clients_corrected)*100:.1f}%)")

# Pour chaque client actif, récupérer toutes ses villes dans df_ventes
df_ventes_villes = df_ventes.groupby('client_id').agg(
    villes_ventes=('ville', lambda x: set(x)),
    nb_villes_diff=('ville', 'nunique'),
    nb_ventes=('ville', 'count')
).reset_index()

# Fusionner avec df_clients_actifs
df_comparaison = df_clients_actifs[['client_id', 'ville', 'derniere_activite']].merge(
    df_ventes_villes,
    on='client_id',
    how='inner'
)

# Vérifier si la ville du client est dans l'ensemble des villes de ses ventes
df_comparaison['ville_trouvee'] = df_comparaison.apply(
    lambda row: row['ville'] in row['villes_ventes'],
    axis=1
)

nb_correct = df_comparaison['ville_trouvee'].sum()
nb_incorrect = (~df_comparaison['ville_trouvee']).sum()
total = len(df_comparaison)

pourcentage_correct = (nb_correct / total * 100) if total > 0 else 0
pourcentage_incorrect = (nb_incorrect / total * 100) if total > 0 else 0

print(f"\n--- Résultats de la comparaison ---")
print(f"Total de clients actifs comparés : {total}")
print(f"✅ Clients avec ville correspondante : {nb_correct} ({pourcentage_correct:.1f}%)")
print(f"❌ Clients avec ville NON trouvée dans leurs ventes : {nb_incorrect} ({pourcentage_incorrect:.1f}%)")

# Statistiques supplémentaires
clients_multivilles = (df_comparaison['nb_villes_diff'] > 1).sum()
print(f"\n📊 Clients ayant commandé depuis plusieurs villes : {clients_multivilles}")

# Afficher les exemples d'incohérence
if nb_incorrect > 0:
    print("\n--- Exemples de clients avec villes incohérentes ---")
    df_incoherents = df_comparaison[~df_comparaison['ville_trouvee']][
        ['client_id', 'ville', 'villes_ventes', 'nb_villes_diff', 'nb_ventes', 'derniere_activite']
    ].head(10)
    display(df_incoherents)



--- ANALYSE CORRESPONDANCE VILLES CLIENT vs VENTES (CLIENTS ACTIFS) ---
Période des ventes : du 2024-10-29 au 2025-10-29

Clients total dans df_clients_corrected : 500
Clients actifs dans la période : 500 (100.0%)

--- Résultats de la comparaison ---
Total de clients actifs comparés : 489
✅ Clients avec ville correspondante : 38 (7.8%)
❌ Clients avec ville NON trouvée dans leurs ventes : 451 (92.2%)

📊 Clients ayant commandé depuis plusieurs villes : 432

--- Exemples de clients avec villes incohérentes ---


,client_id,ville,villes_ventes,nb_villes_diff,nb_ventes,derniere_activite
0,1,Lüe,{Vidal},1,1,2025-01-21
2,3,Oô,"{Duhamel, Sainte Laurencedan, nan, Faivre}",3,3,2025-10-19
3,4,Saint-Eugène,"{Saint Hugues-la-Forêt, Dos Santos-la-Forêt, P...",3,3,2025-07-11
4,5,Bains,"{Lévêque, Saint Eugène, Saint Richardnec}",3,3,2025-10-23
5,6,Bert,"{Hervé, Valette}",2,2,2025-09-09
6,7,Mer,"{Pereira, Sainte Anaïs, Pariss}",3,5,2024-11-27
8,9,Pons,"{Meyer-sur-Pinto, Le Gall, Pariss}",3,3,2025-06-03
9,10,Oô,"{Gosselin, Hamondan, nan, Auger}",3,3,2025-06-29
10,12,NaN,"{Saint ThibaultVille, Pariss, Sanchez}",3,3,2025-07-19
11,13,NaN,"{Vaillantdan, Hebert}",2,2,2024-12-24


In [20]:
print("\n--- ANALYSE CORRESPONDANCE VILLES CLIENT vs VENTES (DERNIÈRE VENTE) ---")

# Convertir la date en datetime
df_ventes['date'] = pd.to_datetime(df_ventes['date'])

# Garder uniquement la dernière vente par client
df_ventes_dernieres = df_ventes.sort_values('date', ascending=False).drop_duplicates(subset='client_id', keep='first')

# Fusionner avec df_clients_corrected
df_comparaison = df_clients_corrected[['client_id', 'ville']].merge(
    df_ventes_dernieres[['client_id', 'ville', 'date']],
    on='client_id',
    how='inner',
    suffixes=('_client', '_vente')
)

# Calculer l'ancienneté de la dernière vente
date_aujourdhui = pd.Timestamp.now()
df_comparaison['jours_depuis_vente'] = (date_aujourdhui - df_comparaison['date']).dt.days

# Comparer les villes
villes_identiques = df_comparaison['ville_client'] == df_comparaison['ville_vente']
nb_correct = villes_identiques.sum()
nb_incorrect = (~villes_identiques).sum()
total = len(df_comparaison)

pourcentage_correct = (nb_correct / total * 100) if total > 0 else 0
pourcentage_incorrect = (nb_incorrect / total * 100) if total > 0 else 0

print(f"Total de clients comparés : {total}")
print(f"✅ Villes correspondantes : {nb_correct} ({pourcentage_correct:.1f}%)")
print(f"❌ Villes différentes : {nb_incorrect} ({pourcentage_incorrect:.1f}%)")

# Analyse des incohérences
if nb_incorrect > 0:
    df_incoherents = df_comparaison[~villes_identiques]

    print(f"\n--- Analyse des {nb_incorrect} incohérences ---")
    print(f"Ancienneté moyenne de la dernière vente : {df_incoherents['jours_depuis_vente'].mean():.0f} jours")
    print(f"Ancienneté médiane : {df_incoherents['jours_depuis_vente'].median():.0f} jours")

    print("\n--- Exemples ---")
    display(df_incoherents[['client_id', 'ville_client', 'ville_vente', 'date', 'jours_depuis_vente']].head(10))



--- ANALYSE CORRESPONDANCE VILLES CLIENT vs VENTES (DERNIÈRE VENTE) ---
Total de clients comparés : 489
✅ Villes correspondantes : 0 (0.0%)
❌ Villes différentes : 489 (100.0%)

--- Analyse des 489 incohérences ---
Ancienneté moyenne de la dernière vente : 123 jours
Ancienneté médiane : 103 jours

--- Exemples ---


,client_id,ville_client,ville_vente,date,jours_depuis_vente
0,1,Lüe,Vidal,2025-03-05,276
1,2,NaN,Boulay,2025-08-15,113
2,3,Oô,Faivre,2025-08-01,127
3,4,Saint-Eugène,Picard-sur-Laine,2025-10-14,53
4,5,Bains,Saint Richardnec,2025-10-16,51
5,6,Bert,Hervé,2025-04-02,248
6,7,Mer,Pariss,2025-06-21,168
7,8,NaN,NaN,2025-10-25,42
8,9,Pons,Meyer-sur-Pinto,2025-04-10,240
9,10,Oô,NaN,2025-04-23,227


In [ ]:
# Créer un nouveau DataFrame pour l'analyse (préservation des données originales)
df_analyse_ventes = df_ventes.copy()

# 1. Inverser le signe des valeurs négatives dans 'chiffre_affaires'
df_analyse_ventes['chiffre_affaires'] = df_analyse_ventes['chiffre_affaires'].abs()

# 2. Convertir la colonne 'date' en format datetime
df_analyse_ventes['date'] = pd.to_datetime(df_analyse_ventes['date'], errors='coerce')

# 3. Extraire le trimestre à partir de la date
df_analyse_ventes['trimestre'] = df_analyse_ventes['date'].dt.to_period('Q')

# 4. Calculer le chiffre d'affaires par trimestre
ca_par_trimestre = df_analyse_ventes.groupby('trimestre')['chiffre_affaires'].agg([
    ('nombre_ventes', 'count'),
    ('ca_total', 'sum')
]).reset_index()

# Afficher les résultats
print("=== Chiffre d'affaires par trimestre ===")
print(ca_par_trimestre)
print(f"\nNombre total de lignes analysées : {len(df_analyse_ventes)}")
print(f"Nombre de valeurs négatives corrigées : {(df_ventes['chiffre_affaires'] < 0).sum()}")

# Analyse croisée trimestre × canal
ca_trimestre_canal = df_analyse_ventes.groupby(['trimestre', 'canal'])['chiffre_affaires'].agg([
    ('nombre_ventes', 'count'),
    ('ca_total', 'sum'),
    ('ca_moyen', 'mean')
]).reset_index()

# Créer un tableau pivot pour une meilleure lisibilité
pivot_ca = ca_trimestre_canal.pivot_table(
    index='trimestre',
    columns='canal',
    values='ca_total',
    fill_value=0
)

print("\n=== CA par trimestre et canal ===")
print(pivot_ca)


=== Chiffre d'affaires par trimestre ===
  trimestre  nombre_ventes   ca_total
0    2024Q4            346   84981.15
1    2025Q1            506  145268.79
2    2025Q2            491  129743.01
3    2025Q3            513  133047.65
4    2025Q4            144   35718.10

Nombre total de lignes analysées : 2000
Nombre de valeurs négatives corrigées : 222

=== CA par trimestre et canal ===
canal      Boutique  Marketplace       Web
trimestre                                 
2024Q4     25656.26      7379.21  51945.68
2025Q1     51301.64     15103.56  78863.59
2025Q2     44018.94     11418.54  74305.53
2025Q3     40393.37     12555.94  80098.34
2025Q4     12284.85      3212.63  20220.62


In [27]:
# Créer un nouveau DataFrame pour l'analyse (préservation des données originales)
df_analyse_ventes = df_ventes.copy()

# 1. Inverser le signe des valeurs négatives dans 'chiffre_affaires'
df_analyse_ventes['chiffre_affaires'] = df_analyse_ventes['chiffre_affaires'].abs()

# 2. Convertir la colonne 'date' en format datetime
df_analyse_ventes['date'] = pd.to_datetime(df_analyse_ventes['date'], errors='coerce')

# 3. Extraire le trimestre et l'année à partir de la date
df_analyse_ventes['trimestre'] = df_analyse_ventes['date'].dt.to_period('Q')
df_analyse_ventes['annee'] = df_analyse_ventes['date'].dt.year

# 4. Calculer le chiffre d'affaires par trimestre
ca_par_trimestre = df_analyse_ventes.groupby('trimestre')['chiffre_affaires'].sum().reset_index()
ca_par_trimestre.columns = ['trimestre', 'ca_total']

# 5. Calculer le chiffre d'affaires pour l'année 2025
ca_2025 = df_analyse_ventes[df_analyse_ventes['annee'] == 2025]['chiffre_affaires'].sum()

# Afficher les résultats
print("=== Chiffre d'affaires par trimestre ===")
print(ca_par_trimestre)

print("\n=== Chiffre d'affaires année 2025 ===")
print(f"CA Total 2025 : {ca_2025:,.2f} €")

print(f"\nNombre total de lignes analysées : {len(df_analyse_ventes)}")
print(f"Nombre de valeurs négatives corrigées : {(df_ventes['chiffre_affaires'] < 0).sum()}")


=== Chiffre d'affaires par trimestre ===
  trimestre   ca_total
0    2024Q4   84981.15
1    2025Q1  145268.79
2    2025Q2  129743.01
3    2025Q3  133047.65
4    2025Q4   35718.10

=== Chiffre d'affaires année 2025 ===
CA Total 2025 : 443,777.55 €

Nombre total de lignes analysées : 2000
Nombre de valeurs négatives corrigées : 222


In [ ]:
# print("\n--- SAUVEGARDE DU FICHIER VENTES AVEC CORRECTIONS ---")

# # Sauvegarder dans le dossier processed
# df_ventes_clean.to_csv('../data/processed/Ventes_Q1_2025_corrected.csv', index=False)